In [6]:
import csv
#잡음신호 가져오기
def csv2list(s):
    file = open(s, 'r')
    csvfile = csv.reader(file)
    lists = []
    for item in csvfile:
        lists.append(item)
    return lists[0]

#신호 발생기
length = 72 * 72 + 1000
caos_num = 3.98
ratio = 0.3
x1 = 0.0437853
x2 = 0.7324354


listr1 = [] #혼돈계 1
listr2 = [] #혼돈계 2
rn = csv2list('noise.txt')

for i in rn:
    k = float(i) #0~1사이의 어떤 신호
    x1 = caos_num * (ratio * k + x1 - x1 * ratio) * (1 - (ratio * k + x1 - x1 * ratio))
    x2 = caos_num * (ratio * k + x2 - x2 * ratio) * (1 - (ratio * k + x2 - x2 * ratio))
    listr1.append(x1)
    listr2.append(x2)

#x1-x2
result = []
for i in range(length):
    result.append(listr1[i]-listr2[i])

a = 0  
#동기화 시작값 탐색  
for i in range(length):
    if result[i] == 0:
        print("동기화 시작 부분: ", i)
        start = i
        a = 1
        break

#동기화가 되지 않을때
if a != 1:
    print("동기화 부분 없음")

    
    
#혼돈신호 가공(복호화배치를 위한)
NoJungBok = [] #가공된 신호
for i in listr1[:72 * 72 + 500]: #500은 중복될 것으로 예상한 개수
    if i not in NoJungBok:
        NoJungBok.append(i)
NoJungBok = NoJungBok[:72*72] #개수에 맞게 슬라이싱

sortedNJB = sorted(NoJungBok)
for i in NoJungBok: #순서 찾기
    NoJungBok[NoJungBok.index(i)] = sortedNJB.index(i) #소수값 대신 순서값 넣어주기

#=========이미지 불러오기=========================
#모듈 임포트
import cv2
import numpy as np

#이미지 불러오기
img = cv2.imread(r'enc.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#==============XOR 복호화 부분========================
chaos = listr1[start+1: start+720+1]#혼돈신호 가져오기

#혼돈신호를 8비트로 변환하기
def chant(num):
    for i in range(256):
        if num >= i / 250 and num < (i+1) / 250:
            return i
for i in chaos:
    chaos[chaos.index(i)] = chant(i)

#XOR 복호화기
for j in range(720):
    for i in range(720):
        img[j, i] = (img[j, i] ^ chaos[i])
cv2.imshow('XOR', img)

#=================스크램블링 복호화 부분===============
#암호화될 이미지를 넣을 이미지 만들기
dec = np.zeros((720,720), dtype=np.uint8)

#이미지 나누기
area = {}
cnt = 0
for i in range(0,720,10):
    for j in range(0, 720, 10):
        area[cnt]=img[j:j+10, i:i+10]
        cnt += 1
#cv2.imshow('h',area[1022])
#이미지 재배치
cnt = 0
for j in range(0,720,10):
    for i in range(0,720,10):
        dec[i:i+10, j:j+10] = area[NoJungBok[cnt]]
        cnt += 1

        
#이미지 출력
#cv2.imshow('h', dec[-0:-10, 0:10])
cv2.imshow('dec', dec)
cv2.waitKey(0)
cv2.destroyAllWindows()

동기화 시작 부분:  391
